    IBM Applied Data Science Capstone Course by Coursera
     Week 5  Report
     Opening an Italian Restaurant in Chicago
    1. Build a dataframe of neighborhoods in Chicago, USA by web scraping using beutifulSoup package. 
    2. Get the geographical coordinates of the localities in chicago using Geocode package
    3. Fetch data for venues for all localities/neighbouhoods using Foursquare API
    4. Explore and cluster the neighborhoods
    5. Select the best cluster to open a new indian restaurant

In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import html5lib
import numpy as np
import geocoder as ge
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import os
import numpy as np

## You can specify any catagory here to cluster on later based on the list of the categories from FourSquare API
restaurant_cuisine = r"Italian Restaurant"


pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)

radius=500
LIMIT=100
CLIENT_ID = 'YWCENNEUMVAI2UDYTKCL03XMDKH4LHVLVMTMAEBUQXFRD1PX' # your Foursquare ID, this has been masked
CLIENT_SECRET = 'EX4USGQYZVPLOR23DG3SVTTFOELQ3EO2TV0H0KXP2SGT5FDI' # youyour Foursquare Secret, this has been masked
VERSION = '20180605' # Foursquare API version

chicago_localities = "https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago"
r = requests.get(chicago_localities)
soup = BeautifulSoup(r.content,'html.parser')
table = soup.find('table')
neighborhoodList = []
dfs = pd.read_html(str(table))


df = pd.DataFrame(dfs[0]).drop(['Community area'],axis=1).reset_index()
# df = df[:10]
print (df)

     index                         Neighborhood
0        0                          Albany Park
1        1                      Altgeld Gardens
2        2                        Andersonville
3        3                       Archer Heights
4        4                        Armour Square
5        5                              Ashburn
6        6                      Ashburn Estates
7        7                       Auburn Gresham
8        8                          Avalon Park
9        9                             Avondale
10      10                     Avondale Gardens
11      11                    Back of the Yards
12      12                      Belmont Central
13      13                      Belmont Gardens
14      14                      Belmont Heights
15      15                      Belmont Terrace
16      16                              Beverly
17      17                         Beverly View
18      18                        Beverly Woods
19      19                             B

 1. create a function get_lat_and_lang to get co ordinates for all 246 neighborhoods 
 2. Draw chicago map using Folium package
 3. get venues for all neighborhoods using FourSquare API
 4. Store al the chicago venues in chicago_venues (around 6339 venues for all 246 localities)

In [8]:

# function to get coordinates
def get_lat_and_lng(neighborhood):
    lat_lng = None
    # loop until you get the coordinates
    while(lat_lng is None):
        g = ge.arcgis('{}, Chicago, USA'.format(neighborhood))
        lat_lng = g.latlng
    return lat_lng

lat_and_land = [ get_lat_and_lng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

df_coordinates = pd.DataFrame(lat_and_land, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coordinates['Latitude']
df['Longitude'] = df_coordinates['Longitude']


address = 'Chicago, USA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago, USA are {}, {}.'.format(latitude, longitude))

map_chicago = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)

folium.ColorMap(map_chicago)

map_chicago.save('map_chicago.html')

def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
                                name,
                                lat,
                                lng,
                                v['venue']['name'],
                                v['venue']['location']['lat'],
                                v['venue']['location']['lng'],
                                v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood Latitude',
                             'Neighborhood Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']

    return(nearby_venues)

chicago_venues = getNearbyVenues(names = df['Neighborhood'],
                                 latitudes = df['Latitude'],
                                 longitudes =  df['Longitude']
                                 )
print(chicago_venues.head())



The geograpical coordinate of Chicago, USA are 41.8755616, -87.6244212.
  Neighborhood  Neighborhood Latitude  Neighborhood Longitude                     Venue  Venue Latitude  Venue Longitude      Venue Category
0  Albany Park               41.96829               -87.72338      Lawrence Fish Market       41.968280       -87.726250  Seafood Restaurant
1  Albany Park               41.96829               -87.72338  Chicago Kalbi Korean BBQ       41.968314       -87.722771   Korean Restaurant
2  Albany Park               41.96829               -87.72338         El Gallo Bravo #6       41.968324       -87.721338  Mexican Restaurant
3  Albany Park               41.96829               -87.72338                 Starbucks       41.968911       -87.728817         Coffee Shop
4  Albany Park               41.96829               -87.72338               Rojo Gusano       41.968425       -87.724549          Taco Place


     1. get all venue cateogries for all localities 
     2.Get of frequency of occurances of italian restaurants in each locality

In [9]:

chicago_restaurant_venues = chicago_venues[chicago_venues['Venue Category'].str.contains("Restaurant")]

## Count venues by each Neighborhood
chicago_venues.groupby('Neighborhood').count()

chicago_restaurant_venues.groupby('Neighborhood').count()

##count unique categories
print('There are {} uniques categories.'.format(len(chicago_venues['Venue Category'].unique())))


# one hot encoding
chicago_onehot = pd.get_dummies(chicago_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
chicago_onehot['Neighborhood'] = chicago_venues['Neighborhood']
chicago_grouped = chicago_onehot.groupby('Neighborhood').mean().reset_index()
chicago_restaurant = chicago_grouped[["Neighborhood",restaurant_cuisine]]

print(chicago_restaurant.head())
print("\n"+ "##############################" + "\n" )
print(chicago_venues['Venue Category'].unique())

There are 359 uniques categories.
      Neighborhood  Italian Restaurant
0      Albany Park            0.000000
1  Altgeld Gardens            0.000000
2    Andersonville            0.035714
3   Archer Heights            0.000000
4    Armour Square            0.043478

##############################

['Seafood Restaurant' 'Korean Restaurant' 'Mexican Restaurant'
 'Coffee Shop' 'Taco Place' 'Discount Store' 'Fried Chicken Joint'
 'Pet Store' 'Dive Bar' 'Pharmacy' 'Mobile Phone Shop' 'Cosmetics Shop'
 'Bank' 'Park' 'Fast Food Restaurant' 'Hookah Bar' 'Bus Station'
 'Indian Restaurant' 'Pizza Place' 'Financial or Legal Service' 'Bakery'
 'Pie Shop' 'Italian Restaurant' 'Thrift / Vintage Store' 'Breakfast Spot'
 'Café' 'Southern / Soul Food Restaurant' 'Lounge' 'Greek Restaurant'
 'Ice Cream Shop' 'Salon / Barbershop' 'Sandwich Place' 'Comic Shop'
 'Grocery Store' 'Whisky Bar' 'New American Restaurant' 'Irish Pub'
 'Miscellaneous Shop' 'Restaurant' 'Burger Joint' 'Gym'
 'Mediterranean Resta

In [ ]:
1.Create 3 clusters for all 246 localities 
2.Distribute all venues into these culsters 

In [10]:
kclusters = 3
chicago_clustering = chicago_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chicago_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
chicago_merged = chicago_restaurant.copy()
chicago_merged["Cluster Labels"] = kmeans.labels_


# merge chicago_merged with chicago to add latitude/longitude for each neighborhood
chicago_merged = chicago_merged.join(df.set_index('Neighborhood'), on='Neighborhood')
chicago_merged.head() # check the last columns!
chicago_merged.sort_values(["Cluster Labels"], inplace=True)


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chicago_merged['Latitude'], chicago_merged['Longitude'], chicago_merged['Neighborhood'], chicago_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

## display clsuter number 1
print(chicago_merged.loc[chicago_merged['Cluster Labels'] == 0])




               Neighborhood  Italian Restaurant  Cluster Labels  index   Latitude  Longitude
153             Oriole Park                 0.0               0    159  41.979030 -87.814430
93   Jackson Park Highlands                 0.0               0     98  41.771420 -87.578940
34                  Chatham                 0.0               0     35  41.741080 -87.613030
87            Humboldt Park                 0.0               0     91  41.899070 -87.719470
46             Douglas Park                 0.0               0     47  41.863150 -87.696890
147                 Oakland                 0.0               0    152  41.824960 -87.605650
214             Vittum Park                 0.0               0    218  41.802090 -87.748570
47                  Dunning                 0.0               0     48  41.952740 -87.796510
48             East Chatham                 0.0               0     50  41.741080 -87.613030
219            West Chatham                 0.0               0    224

In [11]:
print(chicago_merged.loc[chicago_merged['Cluster Labels'] == 1])


                            Neighborhood  Italian Restaurant  Cluster Labels  index   Latitude  Longitude
157                             Parkview            0.153846               1    163  41.950893 -87.724646
167                        Printer's Row            0.011364               1    173  41.873690 -87.629650
158                        Peterson Park            0.000000               1    164  41.985820 -87.728480
174                           River West            0.000000               1    181  41.948787 -87.855967
159                            Pill Hill            0.000000               1    165  41.778639 -87.601627
173                          River North            0.070000               1    180  41.892520 -87.624770
160                               Pilsen            0.000000               1    166  41.857020 -87.657590
161                      Polish Downtown            0.000000               1    167  41.709865 -86.907808
162                       Polish Village      

In [12]:
print(chicago_merged.loc[chicago_merged['Cluster Labels'] == 2])


           Neighborhood  Italian Restaurant  Cluster Labels  index   Latitude  Longitude
233       West Woodlawn                 0.0               2    239  41.718548 -87.594670
212  University Village                 0.0               2    215  41.718529 -87.595898
29             Burnside                 0.0               2     30  41.720310 -87.597500
